In [1]:
# Импорт библиотек
import os
import time

import GTU
import KU_TU
import mat_properties as prop
import numpy as n
import pandas as pd
import SP
from calculate_CCGT_PKM import Calculate_CCGT_PKM_iter
from scipy.optimize import root

# таблица номинального режима
gas_streams0 = pd.read_excel("streams0.xlsx", sheet_name="gas", index_col=0)
water_streams0 = pd.read_excel("streams0.xlsx", sheet_name="water", index_col=0)
GTU_ISO = pd.read_excel("input.xlsx", sheet_name="ISO", index_col=0)
GTU_input = pd.read_excel("input.xlsx", sheet_name="GTU_input", index_col=0)
# рабочая таблица (=номинал в 1 итерации)
gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
water_streams = pd.read_excel("streams.xlsx", sheet_name="water", index_col=0)
# рабочая таблица показателей блоков
heaters = pd.read_excel("blocks.xlsx", sheet_name="heaters", index_col=0)
electric = pd.read_excel("blocks.xlsx", sheet_name="electric", index_col=0)
accumulation = pd.read_excel("blocks.xlsx", sheet_name="accumulation", index_col=0)
############################################################
# Теплосеть и перекидка температуры воздуха
gas_streams.loc["AIR", "T":"P"] = [GTU_input.loc["tair", 1], 0.1]
water_streams.loc["AIR", "T":"P"] = [GTU_input.loc["tair", 1], 0.1]
Tnv = gas_streams.at["AIR", "T"]
water_streams.at["SWIN", "T"] = SP.Tset(Tnv)[1]
water_streams.at["SWOUT", "T"] = SP.Tset(Tnv)[0]
water_streams.at["SWIN-TURB", "T"] = water_streams.at["SWIN", "T"]
water_streams.at["SWIN-TURB", "G"] = water_streams.at["SWIN", "G"]
water_streams.at["SP2-WOUT", "T"] = water_streams.at["SWOUT", "T"]
############################################################

# Основные эффективности оборудования
KPD_PN = 0.8074
KPD_KN = 0.75
KPD_to = 0.99
KPD_SP = 0.99


# Параметры, отвечающие за процесс расчета
Calcmethod = "hybr"
Calctolerance = 10**-2
Maxiterations_KU_TU = 8
Maxiterations_cotel = 5
Maxiterations_turbine = 15

# Параметры режима работы ПГУ
# Расчет для минимума нагрузки
Сalculate_minimum = True


# Расчет для работы с теплофикацией
# Teplo = int(False)
# Отбор пара высокго давления или доля или кг/с
steamVD_fraction_to_turbine = 1


# Состав газов в номинале в ГТУ
gasmix = "Nitrogen*Oxygen*CO2*Water*Argon"
# Считывание рефпропа
RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")
fractiongas0 = list(gas_streams0.loc["GTU-PEVD", "N2":"Ar"])
gas0 = prop.Materials_prop(
    gasmix,
    fractiongas0,
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)
water = prop.Materials_prop(
    "water",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)
# Задание энтальпий газа в номинальном режиме
Temperatures = gas_streams0.loc["GTU-KU":"GPK-out", "T"]
Pressure = gas_streams0.loc["GTU-KU", "P"]
gas_streams0.loc["GTU-KU":"GPK-out", "H"] = list(
    map(lambda x: gas0.p_t(Pressure, x)["h"], Temperatures)
)


#####################Максимов#####################
water_streams.at["SWIN", "H"] = water.p_t(1, water_streams.at["SWIN", "T"])["h"]
water_streams.at["SWOUT", "H"] = water.p_t(1, water_streams.at["SWOUT", "T"])["h"]

# Время зарядки/разрядки, часы
time_ac = 4
# Время ожидания, часы
time_jdat = 12
# Конструкция аккумулятора
constr = {
    "Diametr": 20,
    "kolichestvo": 8,
    "Visota": 20,
    "lambda_min_vata": 0.045,
    "delta_min_vata": 0.01,
}
PKM_zaryad = True
PKM_razryad = False
syngas_streams = pd.read_excel("streams.xlsx", sheet_name="syngas", index_col=0)
##################################################


############################################################
# Задание ГТУ
Gas_turbine = GTU.gtu(GTU_ISO, "GTU-KU")

Iter_pkm = 4
pkm_pgu_tol = 10**-2

arguments_all_it = [
    Maxiterations_KU_TU,
    Maxiterations_cotel,
    Maxiterations_turbine,
    gas_streams0,
    water_streams0,
    GTU_ISO,
    GTU_input,
    gas_streams,
    water_streams,
    heaters,
    electric,
    Gas_turbine,
    gas0,
    water,
    PKM_zaryad,
    PKM_razryad,
    syngas_streams,
    Calcmethod,
    Calctolerance,
    KPD_PN,
    KPD_KN,
    KPD_to,
    KPD_SP,
    steamVD_fraction_to_turbine,
    accumulation,
    time_ac,
    constr,
    time_jdat,
]


########################ОГРАНИЧЕНИЯ НА РАБОТУ ПГУ########################
# ЕСЛИ ТЕПЛОФИКАЦИЯ (из документов по ПГУ-220Т)
# Gк_мин=4,44 кг/с вроде
# Nтурбины мин = 17,6 МВт (примерно 25%)
# Максимальноа давление в отборах СП 0,245, 0,198 МПа (Теплофикационная паровая турбина Т-63/76-8.8 для серии ПГУ-230)
# ДЛЯ ВСЕХ РЕЖИМОВ (Из Трухния по ПГУ-450Т)
# Gвд_мин=25% от номинала
# Gнд_мин = 50% от номинала


########РАСЧЕТ расхода пара на ПКМ из условия заполнения хранилища################
########РАСЧЕТ мощности ГТУ из условия тепловой мощности и ################

################Расчет минимальной нагрузки ГТУ при остальных нормальных условиях############

Max_iterations_minimum = 20

n_GTU_it = [0]
if Сalculate_minimum == True:
    gas_streams.loc["GTU-PEVD", "G"] = gas_streams.loc["GTU-KU", "G"]
    n_GTU = GTU_input.at["n", 1]
    start_time = time.time()
    Delta_min=0
    n_GTU_it.append(round(n_GTU, 5))
    for i in range(Max_iterations_minimum):
        print("n_GTU:", n_GTU_it)
        if i < 4:
            (
                New_iterations_KU_TU,
                New_iterations_cotel,
                New_iterations_turbine,
                New_Iter_pkm,
            ) = (2, 2, 15, 3)
        else:
            (
                New_iterations_KU_TU,
                New_iterations_cotel,
                New_iterations_turbine,
                New_Iter_pkm,
            ) = (
                Maxiterations_KU_TU,
                Maxiterations_cotel,
                Maxiterations_turbine,
                Iter_pkm,
            )

        gas_streams = Calculate_CCGT_PKM_iter(
            arguments_all_it, New_Iter_pkm, pkm_pgu_tol
        )
        Delt_Gcnd = 100
        Delt_Nturb = 100
        Delt_Gcnd = (water_streams.at["INKOND", "G"] - 4.44) / 4.44
        Delt_Nturb = (electric.at["Turbine", "Ni"] - 17.6) / 17.6
        Delt_Gvd = (
            water_streams.at["PEVD-DROSVD", "G"] / water_streams0.at["PEVD-DROSVD", "G"]
            - 0.25
        ) / 0.25
        Delt_Gnd = (
            water_streams.at["PPND-DROSND", "G"] / water_streams0.at["PPND-DROSND", "G"]
            - 0.5
        ) / 0.5
        
        
        
        arguments_all_it[0], arguments_all_it[1], arguments_all_it[2] = (
            New_iterations_KU_TU,
            New_iterations_cotel,
            New_iterations_turbine,
        )
        
        Delta_n_GTU = abs((n_GTU_it[-1] - n_GTU_it[-2]) / n_GTU_it[-1] * 100)
        Delta_min = min(Delt_Gcnd, Delt_Nturb, Delt_Gvd, Delt_Gnd)
        n_GTU = n_GTU - Delta_min / 3
        GTU_input.at["n", 1] = n_GTU
        n_GTU_it.append(round(n_GTU, 5))
        
        
        print(
            f"Время {i+1} итерации расчета мощности ГТУ при ПГУ с ПКМ: --- {round((time.time() - start_time), 1)} сек. ---"
        )
        print(f"Отклонение от ограничения минимальное равно {Delta_min}")
        
        if n_GTU == 1 and Delta_min < 0:
            print("Мощность ГТУ 100% и расход пара все еще слишком мал")

        if abs(Delta_min) < Calctolerance and Delta_n_GTU < Calctolerance:
            arguments_all_it[0], arguments_all_it[1], arguments_all_it[2] = (
                Maxiterations_KU_TU,
                Maxiterations_cotel,
                Maxiterations_turbine,
            )
            n_GTU_it.append(round(n_GTU, 5))
            gas_streams = Calculate_CCGT_PKM_iter(
                arguments_all_it, New_Iter_pkm, pkm_pgu_tol
            )
            print(f"Отклонение от ограничения минимальное равно {Delta_min}")
            print(f"Относительная мощность ГТУ равна {n_GTU}")
            print(
                f"fin минимальная мощность ПГУ:--- {round((time.time() - start_time), 1)} сек. ---"
            )
            print("n_GTU_it", n_GTU_it)
            break
        if i == Max_iterations_minimum - 1:
            print(
                "Достигнуто максимальное количество итераций минимального расхода в ПГУ",
                i + 1,
            )
            print("n_GTU_it", n_GTU_it)

n_GTU: [0, 1]
Тепла от ПКМ не хватает на теплофикацию. Расход на СП: 65.6
Достигнуто максимальное количество итераций котла-утилизатора:2
Достигнуто максимальное количество итераций расхода КУ+ПТУ: 2
Время 1 итерации расчета КУ+ТУ:---  9.2 сек. ---
Достигнуто максимальное количество итераций котла-утилизатора:2
Достигнуто максимальное количество итераций расхода КУ+ПТУ: 2
Время 2 итерации расчета КУ+ТУ:---  20.7 сек. ---
Достигнуто максимальное количество итераций давления КУ+ПТУ: 2
Pnd_it [0.77212, 0.59035, 0.48143]
Pvd_it [9.36936, 6.0131, 4.58998]
fin КУ и ТУ:--- 20.7 сек. ---
Время 1 итерации расчета КУ+ТУ с ПКМ: --- 23.3 сек. ---
Тепла от ПКМ не хватает на теплофикацию. Расход на СП: 123.2
Достигнуто максимальное количество итераций котла-утилизатора:2
Достигнуто максимальное количество итераций расхода КУ+ПТУ: 2
Время 1 итерации расчета КУ+ТУ:---  8.1 сек. ---
Достигнуто максимальное количество итераций котла-утилизатора:2
Достигнуто максимальное количество итераций давления верх

In [2]:
gas_streams

,T,P,H,G,N2,O2,CO2,H2O,Ar
AIR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GTU-KU,528.537683,0.1,943.034860,366.057727,0.780300,0.123700,0.030100,0.059400,0.006500
REF-SMESH,800.000000,0.1,1531.123730,52.828543,0.710321,0.009967,0.090539,0.180531,0.008642
GTU-PEVD,566.186536,0.1,1017.202672,418.886270,0.771289,0.109054,0.037883,0.074998,0.006776
PEVD-IVD,485.961953,0.1,924.166504,418.886270,0.771289,0.109054,0.037883,0.074998,0.006776
IVD-EVD,263.640199,0.1,675.375369,418.886270,0.771289,0.109054,0.037883,0.074998,0.006776
EVD-PPND,194.643158,0.1,600.703774,418.886270,0.771289,0.109054,0.037883,0.074998,0.006776
PPND-IND,192.507709,0.1,598.409158,418.886270,0.771289,0.109054,0.037883,0.074998,0.006776
IND-GPK,142.921197,0.1,545.373324,418.886270,0.771289,0.109054,0.037883,0.074998,0.006776
GPK-out,88.668475,0.1,487.819586,418.886270,0.771289,0.109054,0.037883,0.074998,0.006776


In [3]:
water_streams

,T,P,H,G,S,X
AIR,0.000000,0.100000,NaN,NaN,NaN,NaN
PEVD-DROSVD,495.110886,3.598757,3439.428170,60.279324,NaN,NaN
IVD-PEVD,254.273937,4.271155,2799.379511,60.279324,NaN,NaN
EVD-IVD,250.400828,4.271155,1087.710445,60.279324,NaN,NaN
PEN-EVD,135.846660,4.271155,573.981266,60.279324,NaN,NaN
BND-PEN,133.872692,0.303094,562.924017,60.279324,NaN,NaN
PPND-DROSND,181.195452,0.300048,2827.164917,9.494365,NaN,NaN
IND-PPND,133.872692,0.303094,2725.354573,9.494365,NaN,NaN
GPK-IND,128.974010,0.303094,542.034684,69.773689,NaN,NaN
GPK-REC,128.974010,0.303094,542.034684,82.126725,NaN,NaN


In [4]:
syngas_streams

,T,P,H,G,N2,O2,CO2,Ar,H2O,CH4,H2,CO
REF-COOL,700.0,2.0,3687.984095,45.441849,0.0,0.0,0.063560,0.0,0.536620,0.050040,0.325900,0.023880
COOL-HTS,450.0,2.0,3030.242194,45.441849,0.0,0.0,0.063560,0.0,0.536620,0.050040,0.325900,0.023880
HTS-HTSCOOL,275.0,2.0,2555.053441,45.441849,0.0,0.0,0.086415,0.0,0.513767,0.050042,0.348747,0.001029
HTSCOOL-Separ,100.0,2.0,791.135714,45.441849,0.0,0.0,0.086415,0.0,0.513767,0.050042,0.348747,0.001029
Separ-SGaccum,100.0,2.0,1348.041621,18.074763,0.0,0.0,0.168668,0.0,0.050949,0.097674,0.680700,0.002009
SGaccum-Separacc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Separacc-Sepout,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sepout-COMB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sepout-GTU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AIR-COMP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
electric

,Ni,N,KPDm,KPD
PEN,0.666524,0.873263,0.763257,0.384885
CVD,NaN,NaN,NaN,NaN
CND,NaN,NaN,NaN,NaN
GTU,NaN,85.072341,NaN,28.294523
DK,NaN,6.130392,NaN,NaN
Turbine,17.603803,NaN,NaN,NaN
Tots1,10.586894,NaN,NaN,0.845768
Tots2,3.655471,NaN,NaN,0.837227
Tots3,1.407553,NaN,NaN,0.809827
Tots4,1.953886,NaN,NaN,0.429985


In [6]:
heaters

,Qw,Qg,KPD
PEVD,38581.857817,38971.573552,0.99
IVD,103173.038750,104215.190656,0.99
EVD,30966.116958,31278.906018,0.99
PPND,951.571329,961.183161,0.99
IND,21993.822866,22215.982693,0.99
GPK,23867.385607,24108.470310,0.99
SP2,6.224780,NaN,0.99
SP1,25.142148,NaN,0.99
OD,1.976056,NaN,0.99
Strans,129589.244001,NaN,NaN


In [7]:
accumulation

,Qw,T,V
ASW,NaN,NaN,NaN
PKM,NaN,250.0,10000.0


In [8]:
PKM_zaryad = False
PKM_razryad = True

Maxiterations_KU_TU = 20
Maxiterations_cotel = 5
Maxiterations_turbine = 20

n_GTU = 1
GTU_input.at["n", 1] = n_GTU

arguments_all_it = [
    Maxiterations_KU_TU,
    Maxiterations_cotel,
    Maxiterations_turbine,
    gas_streams0,
    water_streams0,
    GTU_ISO,
    GTU_input,
    gas_streams,
    water_streams,
    heaters,
    electric,
    Gas_turbine,
    gas0,
    water,
    PKM_zaryad,
    PKM_razryad,
    syngas_streams,
    Calcmethod,
    Calctolerance,
    KPD_PN,
    KPD_KN,
    KPD_to,
    KPD_SP,
    steamVD_fraction_to_turbine,
    accumulation,
    time_ac,
    constr,
    time_jdat,
]
New_Iter_pkm = 20
pkm_pgu_tol = 10**-2
CCGT = Calculate_CCGT_PKM_iter(arguments_all_it, New_Iter_pkm, pkm_pgu_tol)

Тепла в ГВТО хватает на теплофикацию
Не сходится баланс камеры сгорания 41.958396943850374 5.31841237716435e-12 41.95839694384506
Рассчитана пиковая ГТУ, расход воздуха: 936.6890019703188
Время 1 итерации расчета КУ+ТУ:---  11.0 сек. ---
Время 2 итерации расчета КУ+ТУ:---  22.9 сек. ---
Достигнуто максимальное количество итераций котла-утилизатора:3
Время 3 итерации расчета КУ+ТУ:---  47.4 сек. ---
Время 4 итерации расчета КУ+ТУ:---  71.5 сек. ---
Время 5 итерации расчета КУ+ТУ:---  84.8 сек. ---
Время 6 итерации расчета КУ+ТУ:---  95.5 сек. ---
Время 7 итерации расчета КУ+ТУ:---  107.0 сек. ---
Время 8 итерации расчета КУ+ТУ:---  116.0 сек. ---
Время 9 итерации расчета КУ+ТУ:---  124.9 сек. ---
Время 10 итерации расчета КУ+ТУ:---  127.3 сек. ---
Время 11 итерации расчета КУ+ТУ:---  128.7 сек. ---
Время 12 итерации расчета КУ+ТУ:---  130.2 сек. ---
Время 13 итерации расчета КУ+ТУ:---  131.7 сек. ---
Расчет КУ+ПТУ окончен.
Максимальная погрешность определения расходов при расчете КУ+ПТУ

In [9]:
GTU_input

,1,2,3,4,5,6
tair,0.0,NaN,NaN,NaN,NaN,NaN
n,1.0,NaN,NaN,NaN,NaN,NaN
Pin,0.0,NaN,NaN,NaN,NaN,NaN
Pout,0.0,NaN,NaN,NaN,NaN,NaN


In [10]:
gas_streams

,T,P,H,G,N2,O2,CO2,H2O,Ar
AIR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GTU-KU,528.537683,0.1,943.034860,366.057727,0.780300,0.123700,0.030100,0.059400,0.006500
REF-SMESH,800.000000,0.1,1531.123730,52.828543,0.710321,0.009967,0.090539,0.180531,0.008642
GTU-PEVD,566.186536,0.1,1017.202672,418.886270,0.771289,0.109054,0.037883,0.074998,0.006776
PEVD-IVD,485.961953,0.1,924.166504,418.886270,0.771289,0.109054,0.037883,0.074998,0.006776
IVD-EVD,263.640199,0.1,675.375369,418.886270,0.771289,0.109054,0.037883,0.074998,0.006776
EVD-PPND,194.643158,0.1,600.703774,418.886270,0.771289,0.109054,0.037883,0.074998,0.006776
PPND-IND,192.507709,0.1,598.409158,418.886270,0.771289,0.109054,0.037883,0.074998,0.006776
IND-GPK,142.921197,0.1,545.373324,418.886270,0.771289,0.109054,0.037883,0.074998,0.006776
GPK-out,88.668475,0.1,487.819586,418.886270,0.771289,0.109054,0.037883,0.074998,0.006776


In [11]:
water_streams

,T,P,H,G,S,X
AIR,0.000000,0.100000,NaN,NaN,NaN,NaN
PEVD-DROSVD,506.193892,8.036576,3414.375409,64.014169,NaN,NaN
IVD-PEVD,298.424368,8.398891,2752.612556,64.014169,NaN,NaN
EVD-IVD,297.052530,8.398891,1328.413491,64.014169,NaN,NaN
PEN-EVD,165.613228,8.398891,704.284783,64.014169,NaN,NaN
BND-PEN,164.196297,0.687080,693.732680,64.014169,NaN,NaN
PPND-DROSND,208.815256,0.681290,2865.827401,15.222004,NaN,NaN
IND-PPND,164.196297,0.687080,2761.964581,15.222004,NaN,NaN
GPK-IND,162.363698,0.687080,685.769184,79.236173,NaN,NaN
GPK-REC,162.363698,0.687080,685.769184,89.383366,NaN,NaN


In [12]:
syngas_streams

,T,P,H,G,N2,O2,CO2,Ar,H2O,CH4,H2,CO
REF-COOL,700.000000,2.0,3687.984095,45.441849,0.000000,0.000000,0.063560,0.000000,0.536620,0.050040,0.325900,0.023880
COOL-HTS,450.000000,2.0,3030.242194,45.441849,0.000000,0.000000,0.063560,0.000000,0.536620,0.050040,0.325900,0.023880
HTS-HTSCOOL,275.000000,2.0,2555.053441,45.441849,0.000000,0.000000,0.086415,0.000000,0.513767,0.050042,0.348747,0.001029
HTSCOOL-Separ,100.000000,2.0,791.135714,45.441849,0.000000,0.000000,0.086415,0.000000,0.513767,0.050042,0.348747,0.001029
Separ-SGaccum,100.000000,2.0,1348.041621,18.074763,0.000000,0.000000,0.168668,0.000000,0.050949,0.097674,0.680700,0.002009
SGaccum-Separacc,5.703306,2.0,1075.096789,365775.995050,0.000000,0.000000,0.168738,0.000000,0.050357,0.097508,0.681387,0.002009
Separacc-Sepout,5.703306,2.0,957.542925,336452.977654,0.000000,0.000000,0.177686,0.000000,0.000000,0.102679,0.717520,0.002115
Sepout-COMB,5.703306,2.0,957.542925,22.283679,0.000000,0.000000,0.177686,0.000000,0.000000,0.102679,0.717520,0.002115
Sepout-GTU,5.703306,2.0,957.542925,1.081111,0.000000,0.000000,0.177686,0.000000,0.000000,0.102679,0.717520,0.002115
AIR-COMP,15.000000,0.1,293.187169,936.689002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
electric

,Ni,N,KPDm,KPD
PEN,0.675484,0.700935,0.963690,0.807400
CVD,NaN,NaN,NaN,NaN
CND,NaN,NaN,NaN,NaN
GTU,NaN,164.304600,NaN,34.012097
DK,NaN,6.298505,NaN,NaN
Turbine,74.362793,NaN,NaN,NaN
Tots1,34.506138,NaN,NaN,0.849306
Tots2,9.371839,NaN,NaN,0.837238
Tots3,5.477841,NaN,NaN,0.814967
Tots4,25.006975,NaN,NaN,0.848781


In [14]:
heaters

,Qw,Qg,KPD
PEVD,42361.482865,42789.376631,0.99
IVD,91169.543813,92090.448296,0.99
EVD,39953.753354,40357.326620,0.99
PPND,1580.962967,1596.932290,0.99
IND,32113.137296,32437.512421,0.99
GPK,38794.941366,39186.809461,0.99
SP2,0.000000,NaN,0.00
SP1,0.000000,NaN,0.00
OD,0.000000,NaN,0.00
Strans,0.000000,NaN,NaN


In [15]:
accumulation

,Qw,T,V
ASW,NaN,NaN,NaN
PKM,8.943377e+09,5.703306,10000.0
